## Simulator (fleet of uavs)

In [4]:
from simulator import Simulator
from simulator.config import Color
from simulator.helpers import clean
from simulator.helpers.coordinates import ENUPose, GRAPose
from simulator.planner import AutoPlan, GuidedPlan, Plan
from simulator.visualizer import (
    QGC,
    Gazebo,
    GazMarker,
    NoVisualizer,
    QGCMarker,
    SimVehicle,
)

clean()

## Simulation Positions

In [5]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=0) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (0., 15., 0., 0.),
    (15., 0., 0., 0),
    (5., -20., 0., 30.),
    (-15., -15., 0., 0),
    (-15., 0., 0., 45),

])

## Create Vehicles

In [6]:
sysids = [1,2,3,4,5]
colors=[
    Color.BLUE,
    Color.GREEN,
    Color.BLACK,
    Color.ORANGE,
    Color.RED,
]
side_len = 5
alt = 5

vehs:list[SimVehicle] = []

for sysid, base_home, color in zip(sysids, base_homes, colors):
    auto_plan = AutoPlan.square_traj(
        side_len=side_len,
        alt=alt,
        name="simple_auto_plan",
        sysid=sysid,
        gra_origin=gra_origin,
        relative_home=base_home,
    )
    
    guided_plan = GuidedPlan.square_traj(
                                    side_len=side_len, 
                                    alt=alt,
                                    enu_origin=enu_origin,
                                    relative_home=base_home)


    veh = SimVehicle.from_relative(
        sysid=sysid,
        gcs_name=f'Multicolor_{"".join([color.emoji for color in colors])}',
        plan=auto_plan,
        color=color,
        enu_origin=enu_origin,
        relative_home=base_home,
        relative_path=Plan.create_square_path(side_len=side_len, alt=alt),
        model="iris",
    )
    
    vehs.append(veh)
    
    

## Visualizer

### Gazebo

In [7]:
gaz= Gazebo(gra_origin,world_path="simulator/visualizer/gazebo/worlds/runway.world")
origin_gaz = GazMarker(name="origin",
                    group="origin",
                    pos=enu_origin.unpose(),
                    color=Color.WHITE)
gaz.markers.append(origin_gaz)

### QGroundControl

In [8]:
qgc= QGC(gra_origin)
origin_qgc = QGCMarker(name="origin",
                pos=gra_origin.unpose(),
                color=Color.WHITE)
qgc.markers.append(origin_qgc)

### No Visualizer

In [9]:
novis = NoVisualizer(gra_origin)

## Simulator

In [10]:
simulator = Simulator(
	visualizer=qgc,
	terminals=['gcs'],
	verbose=1,
)
for veh in vehs:
    simulator.add_vehicle(veh)

simulator.show()

## Run

In [11]:
orac = simulator.launch()
orac.run()

05:02:38 - Oracle ⚪ - INFO - 🚀 GCS Multicolor_🟦🟩⬛🟧🟥 launched (PID 272793)
05:02:38 - Oracle ⚪ - INFO - 🗺️  QGroundControl launched for 2D visualization — simulation powered by ArduPilot SITL.
05:02:38 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 5 vehicles and 1 GCSs
05:04:06 - Oracle ⚪ - INFO - UAV 3 completed mission and exited
05:04:07 - Oracle ⚪ - INFO - UAV 5 completed mission and exited
05:04:09 - Oracle ⚪ - INFO - UAV 4 completed mission and exited
05:04:09 - Oracle ⚪ - INFO - UAV 2 completed mission and exited
05:04:10 - Oracle ⚪ - INFO - Received DONE from GCS Multicolor_🟦🟩⬛🟧🟥
05:04:10 - Oracle ⚪ - INFO - UAV 1 completed mission and exited
05:04:10 - Oracle ⚪ - INFO - ✅ All GCS threads completed
05:04:10 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
